# Importando librerías

In [276]:
import krakenex
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
import datetime
import time
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
import requests
import streamlit as st
import json

# Identificando pares disponibles

In [183]:
assets_names = pd.read_csv("asset_names.csv")

# Definiendo API de Kraken

In [185]:
kraken = krakenex.API()

# Obteniendo hora del servidor

In [186]:
def serverTime():
    server_time = kraken.query_public('Time')
    if server_time['error']:
        return server_time['error']
    else:
        return server_time['result']

# Verificando estado del sistema

In [188]:
def systemStatus():
    system_status = kraken.query_public('SystemStatus')
    if system_status['error']:
        print(system_status['error'])
    else:
        return system_status['result']

# Definiendo criptomonédas a analizar y tipos de cambio

In [190]:
assets = list(kraken.query_public('Assets', {'asset':'XBT, ETH'})['result'].keys())

In [191]:
pairs = kraken.query_public('AssetPairs')

In [192]:
pair_lines = []
for key, value in pairs['result'].items():
    pair_lines.append(value)

In [193]:
columns = list(pd.DataFrame.from_records(pairs['result']).index)

In [194]:
df_pairs = pd.DataFrame.from_records(
    pair_lines, 
    columns=columns)

In [195]:
df_pairs['valid_asset']=df_pairs.base.isin(assets)

In [196]:
df_pairs = df_pairs[df_pairs['valid_asset'] == True]

In [197]:
assets_names.drop('type', axis=1, inplace=True)

In [198]:
assets_names = assets_names.rename(columns={'code': 'base', 'name': 'base_name'})
quote_names = assets_names.rename(columns={'base': 'quote', 'base_name': 'quote_name'})


In [200]:
df_pairs = df_pairs.merge(assets_names, how='left', on='base')
df_pairs = df_pairs.merge(quote_names, how='left', on='quote')

In [271]:
assets = df_pairs.groupby(by=['base', 'base_name']).count().reset_index()[['base_name', 'base']].to_dict('split')['data']

In [289]:
df_pairs['base_quote'] = df_pairs['base'] + df_pairs['quote']

In [305]:
list(df_pairs['base_quote'])

['XETHAED',
 'XETHZAUD',
 'XETHCHF',
 'XETHDAI',
 'XETHUSDC',
 'XETHUSDT',
 'XXBTAED',
 'XXBTZAUD',
 'XXBTCHF',
 'XXBTDAI',
 'XXBTUSDC',
 'XXBTUSDT',
 'XETHXXBT',
 'XETHZCAD',
 'XETHZEUR',
 'XETHZGBP',
 'XETHZJPY',
 'XETHZUSD',
 'XXBTZCAD',
 'XXBTZEUR',
 'XXBTZGBP',
 'XXBTZJPY',
 'XXBTZUSD']

In [301]:
asset = assets[0][1]

In [302]:
df_pairs[df_pairs['base'] == asset]

,altname,wsname,aclass_base,base,aclass_quote,quote,lot,cost_decimals,pair_decimals,lot_decimals,...,ordermin,costmin,tick_size,status,long_position_limit,short_position_limit,valid_asset,base_name,quote_name,base_quote
0,ETHAED,ETH/AED,currency,XETH,currency,AED,unit,5,1,8,...,0.01,NaN,0.1,cancel_only,NaN,NaN,True,Ethereum,NaN,XETHAED
1,ETHAUD,ETH/AUD,currency,XETH,currency,ZAUD,unit,6,2,8,...,0.01,NaN,0.01,online,37.0,37.0,True,Ethereum,Australian Dollar,XETHZAUD
2,ETHCHF,ETH/CHF,currency,XETH,currency,CHF,unit,5,2,8,...,0.01,NaN,0.01,online,NaN,NaN,True,Ethereum,NaN,XETHCHF
3,ETHDAI,ETH/DAI,currency,XETH,currency,DAI,unit,6,3,8,...,0.01,NaN,0.001,online,NaN,NaN,True,Ethereum,Dai,XETHDAI
4,ETHUSDC,ETH/USDC,currency,XETH,currency,USDC,unit,6,2,8,...,0.01,0.5,0.01,online,175.0,250.0,True,Ethereum,USD Coin,XETHUSDC
5,ETHUSDT,ETH/USDT,currency,XETH,currency,USDT,unit,5,2,8,...,0.01,0.5,0.01,online,250.0,250.0,True,Ethereum,Tether,XETHUSDT
12,ETHXBT,ETH/XBT,currency,XETH,currency,XXBT,unit,6,5,8,...,0.01,0.00002,0.00001,online,250.0,400.0,True,Ethereum,Bitcoin,XETHXXBT
13,ETHCAD,ETH/CAD,currency,XETH,currency,ZCAD,unit,5,2,8,...,0.01,NaN,0.01,online,NaN,NaN,True,Ethereum,Canadian Dollar,XETHZCAD
14,ETHEUR,ETH/EUR,currency,XETH,currency,ZEUR,unit,5,2,8,...,0.01,0.45,0.01,online,1000.0,1000.0,True,Ethereum,Euro,XETHZEUR
15,ETHGBP,ETH/GBP,currency,XETH,currency,ZGBP,unit,5,2,8,...,0.01,0.43,0.01,online,80.0,80.0,True,Ethereum,Great British Pound,XETHZGBP


In [291]:
df_pairs[df_pairs['base_quote'] == ]

['Ethereum', 'XETH']

# Definiendo funciones

In [310]:
new_data = kraken.query_public(
        'Trades', 
        {
            'pair': df_pairs['base_quote']
        }
    )

In [313]:
new_data['result']

{'XXBTZUSD': [['16569.90000',
   '0.00013219',
   1669465567.4060082,
   'b',
   'l',
   '',
   53519446],
  ['16569.90000', '0.14126029', 1669465570.165722, 'b', 'm', '', 53519447],
  ['16569.90000', '0.00010329', 1669465570.166166, 'b', 'm', '', 53519448],
  ['16571.40000', '0.80642336', 1669465570.166226, 'b', 'm', '', 53519449],
  ['16574.60000', '1.49967925', 1669465570.1662743, 'b', 'm', '', 53519450],
  ['16574.90000', '0.37000000', 1669465570.1663496, 'b', 'm', '', 53519451],
  ['16575.20000', '0.06512310', 1669465570.166486, 'b', 'm', '', 53519452],
  ['16575.30000', '0.50000000', 1669465570.1666152, 'b', 'm', '', 53519453],
  ['16575.40000', '0.27081369', 1669465570.1666827, 'b', 'm', '', 53519454],
  ['16575.40000', '0.36265762', 1669465570.1667619, 'b', 'm', '', 53519455],
  ['16575.50000', '1.04852851', 1669465570.1668575, 'b', 'm', '', 53519456],
  ['16577.40000', '0.08870000', 1669465570.1671026, 'b', 'm', '', 53519457],
  ['16577.80000', '0.48806976', 1669465570.167176,

## Función para extraer los datos

In [221]:
def getData(start_date, end_date, old_data):
    # print(datetime.datetime.utcfromtimestamp(start_date), datetime.datetime.utcfromtimestamp(end_date))
    new_data = kraken.query_public(
        'Trades', 
        {
            'pair': asset_selected.value + quote_selected.value,
            'since': int(start_date)
        }
    )

    if new_data['error']:
        print(new_data['error'])
    else:
        new_data = buildDf(new_data['result'][asset_selected.value + quote_selected.value])

        if old_data is None:
            pass 
        else:
            new_data = pd.concat([old_data, new_data], axis=0)
            
        new_start_date = new_data['time'].iloc[-1]
        
        if new_start_date >= end_date:
            pass
        else:
            time.sleep(1.8)
            new_data = getData(new_start_date, end_date, new_data)

        return new_data



# Workflow

In [227]:
end_date = serverTime()
start_date = defineTimeFrames(datetime.datetime.utcfromtimestamp(end_date['unixtime']))
start_date = transformDatetimeToEpohc(start_date['day']) # Definir serie temporal (hora, día, mes o año)
data = getData(start_date, end_date['unixtime'], None)
# data = buildDf(data)
# data = cleaningData(data)
# data


## Función para constriuir el Data Frame

In [17]:
def buildDf(data):
    df = pd.DataFrame.from_records(
        data,
        columns=[
            'price',
            'volume',
            'time',
            'buy/sell',
            'market/limit',
            'miscellaneous',
            '?'
        ]
    )

    return df


## Función de limpieza de datos

In [19]:
def cleaningData(df):

    time_vars = ['time']
    float_vars = ['price', 'volume']

    for var in time_vars:
        df[var] = pd.to_datetime(df[var], unit='s')

    for var in float_vars:
        df[var] = pd.to_numeric(df[var], errors='coerce')

    return df


# Función para definir la serie temporal del análisis

In [29]:
def defineTimeFrames(server_time):
    # now = datetime.datetime.now()
    hour = server_time - datetime.timedelta(hours=1)
    day = server_time - datetime.timedelta(days=1)
    week = server_time - relativedelta(weeks=1)
    month = server_time - relativedelta(month=1)
    year = server_time - relativedelta(years=1)
    return {'hour':hour, 'day': day, 'week':week, 'month': month, 'year': year}


# Función para transoformar datetime a epoch

In [24]:
def transformDatetimeToEpohc(since):
    return int(time.mktime((since).timetuple()))

## Función generar media móvil

In [147]:
def calculateMovingAverage(df):

    df['SMA25'] = df['price'].rolling(25).mean()

    return df


## Función generar RSI

In [148]:

def calculateRsi(df, periods=14, ema=True):
    """
    Returns a pd.Series with the relative strength index.
    """
    close_delta = df['price'].diff()

    # Make two series: one for lower closes and one for higher closes
    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)

    if ema == True:
        # Use exponential moving average
        ma_up = up.ewm(com=periods - 1, adjust=True,
                       min_periods=periods).mean()
        ma_down = down.ewm(com=periods - 1, adjust=True,
                           min_periods=periods).mean()
    else:
        # Use simple moving average
        ma_up = up.rolling(window=periods, adjust=False).mean()
        ma_down = down.rolling(window=periods, adjust=False).mean()

    rsi = ma_up / ma_down
    rsi = 100 - (100/(1 + rsi))

    df['RSI'] = rsi
    return df


## Función generar indicadores

In [149]:
def calculateIndicators(df):

    # Identificando todos los pares disponibles en la extracción
    pairs = df['pair_name'].unique()

    # Calculado los indicadores para cada par y almacenando en un data frame independiente
    list_of_dfs = []
    for pair in pairs:
        df_pair = df[df['pair_name'] == pair]

        df_pair = calculateMovingAverage(df_pair)
        df_pair = calculateRsi(df_pair)
        df_pair['SMA25'] = df_pair['close'].rolling(25).mean()

        list_of_dfs.append(df_pair)

    # Uniendo todos los data frames en un solo data frame resultante
    # new_df = pd.DataFrame()
    dfs = [df.reset_index(drop=True) for df in list_of_dfs]
    return pd.concat(dfs, axis=0)
    
    # for x in list_of_dfs:
    #     new_df = new_df.append(x)
    # return new_df


# Extrayendo datos

In [150]:
data = getData()

## Validando errores

In [151]:
for x in data:
    if list(x.values())[0]:
        print(f'Se ha producido el siguiente error: {list(x.values())[0][0]}')
        break
    else:
        print('Datos cargados correctamente')

Datos cargados correctamente
Datos cargados correctamente
Datos cargados correctamente
Datos cargados correctamente


# Construyendo Data Frame

In [152]:
df = buildDf(data)
df = df.reset_index(drop=True)


# Limpiando datos

In [153]:
df = cleaningData(df)


# Calculado indicadores

In [154]:
df = calculateIndicators(df)


/var/folders/2y/tls0s71d3llbk2fk7vt0mgc80000gn/T/ipykernel_11347/2824284413.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SMA25'] = df['close'].rolling(25).mean()
/var/folders/2y/tls0s71d3llbk2fk7vt0mgc80000gn/T/ipykernel_11347/3984806358.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RSI'] = rsi
/var/folders/2y/tls0s71d3llbk2fk7vt0mgc80000gn/T/ipykernel_11347/3206932739.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [155]:
df

,time,open,high,low,close,vwap,volume,count,pair_name,SMA25,RSI
0,2022-11-19 12:15:00,16643.80,16645.00,16643.80,16645.00,16644.90,0.160500,7,XXBTZUSD,NaN,NaN
1,2022-11-19 12:16:00,16645.00,16645.00,16645.00,16645.00,16645.00,1.155036,5,XXBTZUSD,NaN,NaN
2,2022-11-19 12:17:00,16645.00,16645.00,16645.00,16645.00,0.00,0.000000,0,XXBTZUSD,NaN,NaN
3,2022-11-19 12:18:00,16645.00,16645.00,16644.90,16645.00,16644.90,0.041627,7,XXBTZUSD,NaN,NaN
4,2022-11-19 12:19:00,16644.90,16645.00,16644.90,16645.00,16644.90,0.015926,8,XXBTZUSD,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
715,2022-11-20 00:10:00,1179.14,1179.35,1179.14,1179.35,1179.31,0.185709,3,XETHZEUR,1178.1776,58.676666
716,2022-11-20 00:11:00,1179.35,1179.35,1179.35,1179.35,0.00,0.000000,0,XETHZEUR,1178.2308,58.676666
717,2022-11-20 00:12:00,1177.96,1177.96,1177.17,1177.17,1177.35,0.169131,3,XETHZEUR,1178.1944,36.962234
718,2022-11-20 00:13:00,1177.59,1177.59,1177.59,1177.59,1177.59,0.195149,1,XETHZEUR,1178.1748,41.457273


# Gráficos

In [156]:
df_test = df[df['pair_name'] == 'XXBTZUSD']


## Gráfico precio

In [157]:
hovertext = []
for i in range(len(df_test['open'])):
    hovertext.append('<br>Precio: '+str(df_test['close'][i]))

fig = go.Figure(data=go.Ohlc(x=df_test['time'],
                open=df_test['open'],
                high=df_test['high'],
                low=df_test['low'],
                close=df_test['close'],
                text=hovertext,
                hoverinfo='text'))

fig.update_layout(
    title='Precio histórico. Últimos 720 periodos',
    yaxis_title='Precio del par XXBTZUSD'
)

fig.show()


## Gráfico média móvil

In [ ]:
hovertext = []
for i in range(len(df_test['open'])):
    hovertext.append('<br>Precio: '+str(df_test['close'][i]))

fig = go.Figure(data=[go.Ohlc(x=df_test['time'],
                open=df_test['open'],
                high=df_test['high'],
                low=df_test['low'],
                close=df_test['close'],
                text=hovertext,
                hoverinfo='text',
                name='Precio'),
    go.Scatter(
    x=df_test['time'],
    y=df_test['SMA25'],
    line=dict(color='blue',
              width=1),
    name='Media móvil 25 períodos')
])

fig.update_layout(
    title='Media móvil. Últimos 720 periodos',
    yaxis_title='Precio del par XXBTZUSD'
)

fig.show()


## Gráfico RSI

In [ ]:
hovertext = []
for i in range(len(df_test['open'])):
    hovertext.append('<br>Precio: '+str(df_test['close'][i]))
layout = {
    "yaxis": {"domain": [0, 0.33]},
    "yaxis2": {"domain": [0.33, 1]}
}
fig = go.Figure(data=[go.Ohlc(
    x=df_test['time'],
    open=df_test['open'],
    high=df_test['high'],
    low=df_test['low'],
    close=df_test['close'],
    text=hovertext,
    hoverinfo='text',
    name='Precio',
    yaxis='y2'),

    go.Scatter(
    x=df_test['time'],
    y=df_test['RSI'],
    line=dict(color='purple',
              width=1),
    name='Índice de fortaleza realativa',
    yaxis="y"),

],
    layout=layout)

fig.update_layout(
    title='Índice de fortaleza relativa (RSI). Últimos 720 periodos',
    yaxis_title='Precio del par XXBTZUSD',
    showlegend=False,
    xaxis_rangeslider_visible=False

)

fig.show()
